In [1]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd

from helper import *

main = pd.read_pickle("../Data/clean_main.df")

In [2]:
# PROCESS:
# Assume that all average times are worth 80 rating, avg
# Take the difference between the average time and the recorded time, avg - act = diff
# Get the percent that the difference represents of the avg time, pctDiff = (diff / avg) * 100
# Add this value (including sign, high finish times than avg should decrease rating below 80)

# Get an equation that will convert from time to rating -> DONE


# NOTE:
# Should find par times for track, distance and get these ratings. 
# Then, should get the average ratings for each class. ONLY USING THE FAST TRACK TIMES (use going to do this)
# Because rating take into account distance, these are useful to apply to any distance

In [3]:
# TRACK VARIANTS
# Find the average speed rating of each race on that track that day
# Take the difference between the avg speed rating and the par class speed rating
# The avgerage difference is the track variant for that day

In [ ]:
def getAvgs(data, data2):
    
    venues = np.unique(data["venue"])
    s = 0.
    for v in venues:
        vData = data.loc[data["venue"]==v]
        vData2 = data2.loc[data2["venue"]==v]
        distances = np.unique(vData["distance"])
        
        for d in distances:
            dData = vData.loc[vData["distance"]==d]
            dData2 = vData2.loc[vData2["distance"]==d]

            # Average finish time difference
            avgTime = np.mean(dData["finish_time"]) - np.mean(dData2["finish_time"])
            s += abs(avgTime)
            print(v, d, avgTime)
    
    print(s)
    
getAvgs(main[:10000], main)

In [ ]:
# Save averages as a dictionary
date = pd.to_datetime("1998-07-19")
data = main[main.date < date]
model_data = main[main.date >= date]
print(len(data))


samples = len(data)
dic = {"samples":samples}

venues = np.unique(data["venue"])
s = 0.
for v in venues:
    vData = data.loc[data["venue"]==v]
    surfaces = np.unique(vData["surface"])
    
    for s in surfaces:
        sData = vData.loc[data["surface"]==s]
        distances = np.unique(sData["distance"])

        for d in distances:
            dData = vData.loc[vData["distance"]==d]

            # Average finish time difference
            avgTime = np.mean(dData["finish_time"])
            string = v + " " + str(s) + " " + str(d)
            dic[string] = avgTime

In [ ]:
# Save
np.save('averages.npy', dic)
#data.to_pickle("avg_data.df")
#model_data.to_pickle("model_data.df")

# Load
#read_dictionary = np.load('averages.npy',allow_pickle='TRUE').item()

In [ ]:
# DO THE SAME THING FOR CLASSES
splitDate = pd.to_datetime("1998-07-19")
data = main[main.date < splitDate]

# Get only fast tracks
fast_tracks = ["GOOD", "GOOD TO FIRM", "FAST"]
data = data.loc[data.going.isin(fast_tracks)]

# Get class avgs
classAvgs = {}
classes = np.unique(main["race_class"])
for c in classes:
    
    races = data.loc[data.race_class == c]
    
    avgRating = races["speed_rating_1"].mean()
    classAvgs[c] = avgRating

print(classAvgs)
np.save('class_averages.npy', classAvgs)

In [4]:
# Add speed ratings to each run in the model data
def addRawSpeedRatings(data):
    data = data.copy()
    dataSize = len(data)
    ratings = np.zeros(dataSize)
    
    for i in range(dataSize):
        printProgress(i, dataSize)
        
        run = data.iloc[i, :]
        venue, surface, distance, time = run["venue"], run["surface"], run["distance"], run["finish_time"]
        
        rating = speed2rating(time, distance, venue, surface)
        ratings[i] = rating
        
    data["speed_rating_1"] = ratings
    
    return data


In [ ]:
main1 = addRawSpeedRatings(main)
main1.to_pickle("../Data/clean_main.df")

0.00% Complete ...
1.31% Complete ...
2.63% Complete ...
3.94% Complete ...
5.25% Complete ...
6.57% Complete ...
7.88% Complete ...
9.20% Complete ...
10.51% Complete ...
11.82% Complete ...
13.14% Complete ...
14.45% Complete ...
15.76% Complete ...
17.08% Complete ...
18.39% Complete ...
19.71% Complete ...
21.02% Complete ...
22.33% Complete ...
23.65% Complete ...
24.96% Complete ...
26.27% Complete ...
27.59% Complete ...
28.90% Complete ...
30.22% Complete ...
31.53% Complete ...
32.84% Complete ...
34.16% Complete ...
35.47% Complete ...
36.78% Complete ...
38.10% Complete ...
39.41% Complete ...


In [ ]:
# Now we create a track variant based on this

dataSize = len(main)
track_vars = np.zeros(dataSize)
dates = np.unique(main["date"])
venues = np.unique(main["venue"])
count = 0
for date in dates:
    printProgress(count, len(dates), jump=100)
    count += 1
    for venue in venues:
        races = main.loc[(main.date == date) & (main.venue == venue)]
        raceI = races.index
        race_ids = np.unique(races.race_id)
        if len(race_ids)==0:
            continue
        diffs = 0.
    
        for race_id in race_ids:
            race = races.loc[races.race_id==race_id]
            c = race.iloc[0,:]["race_class"]
        
            raceAvg = race["speed_rating_1"].mean()
            classAvg = CLASS_AVGS[c]
            diffs += classAvg - raceAvg
        
            if race.loc[race.race_class==c].shape[0] != race.shape[0]:
                print("ISSUE")
        
        trackVar = diffs / len(race_ids)

        for i in raceI:
            track_vars[i] = trackVar
    
main["track_variant"] = track_vars

main.to_pickle("../Data/clean_main.df")

In [ ]:
# Add adjusted speed ratings (speed_rating)
main["speed_rating"] = main.apply(lambda x: x.speed_rating_1 + x.track_variant, axis=1)
main.to_pickle("../Data/clean_main.df")